# OpenAI Experiments with RuMedBench

In [196]:
from config import OPENAI_TOKEN  # private API token

import os
import openai
import config
import pandas as pd
import time
import json
from tqdm.auto import tqdm, trange

# OpenAI access

In [171]:
# test of access
openai.api_key = OPENAI_TOKEN
response = openai.Completion.create(
engine="davinci",
prompt="Blog topics dealing with daily life living on Mars\n\n1.",
temperature=0.3,
max_tokens=64,
top_p=1,
frequency_penalty=0.5,
presence_penalty=0)
print(response)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " The Martian Atmosphere\n\n2. The Martian Climate\n\n3. The Martian Geology\n\n4. The Martian Gravity and the Red Planet's Surface Gravity\n\n5. The Martian Moon Phobos, the Moons of Mars, and Mars' Natural Satellites, Deimos and Phobos\n\n"
    }
  ],
  "created": 1676106446,
  "id": "cmpl-6ig8sw1Gn0evEvdKJLZnNtftaKGi4",
  "model": "davinci",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 64,
    "prompt_tokens": 13,
    "total_tokens": 77
  }
}


In [275]:
class OpenAI():
    """sends requests to openai API with maintaining request rate"""
    
    def __init__(self, engine='davinci',
                request_interval=3.1):
        self.engine=engine
        self.times = []
        self.time_queue_len = 5  # len of queue for keeping last request times
        self.request_interval = 3.1  # min interval between requests

    def wait_ok(self):
        """waits until request imterval over time_queue_len is satisfied"""
        if len(self.times) >= self.time_queue_len:
            while time.time() - self.times[0] < self.time_queue_len * self.request_interval:
                time.sleep(0.1)
        self.times.append(round(time.time(), 2))
        self.times = self.times[-self.time_queue_len:]
        
    def __call__(self, prompt):
        self.wait_ok()
        r = openai.Completion.create(
            engine=self.engine,
            prompt=prompt,
            temperature=0.3,
            max_tokens=64,
            top_p=1,
            frequency_penalty=0.5,
            presence_penalty=0)
        
        return r['choices'][0]['text']

# RuMedTest experiments

## Loading source for pre-tests
Источник: тест по терапии для интернов СОГМА https://geetest.ru/tests/terapiya_(dlya_internov)_sogma_

In [169]:
xf = "questions/sogma_test_complete.xml"  # pre-downloaded from link above

with open(xf, 'rb') as f:
    xx = f.read()

In [14]:
xx[:10]

['<?xml version="1.0" encoding="utf-8"?>\n',
 '<geetest>\n',
 '<test>\n',
 '\n',
 '<name>Терапия (для интернов) СОГМА </name>\n',
 '<give> терапии (для интернов) СОГМА </give>\n',
 '<category>Медицина</category>\n',
 '<isThemes>1</isThemes>\n',
 '<description></description>\n',
 '\n']

In [19]:
dd = xmltodict.parse(xx)
len(dd)

In [26]:
dd['geetest']['test']['questions']['question'].__len__()

770

In [28]:
q = dd['geetest']['test']['questions']['question'][0]

In [31]:
q['text']

'Везикулярное дыхание выслушивается на'

In [56]:
q

{'@id': '22224',
 '@num': '73',
 '@idTest': '84',
 '@idTheme': '2',
 '@ans': '',
 '@idTicket': '0',
 '@numTicket': '0',
 'text': 'При пневмонии назначают следующие лекарственные средства: \n\t1. этиотропные; \n\t2. отхаркивающие; \n\t3. бронхоспазмолитические; \n\t4. иммуномодулирующие',
 'answers': {'answer': [{'@id': '99677',
    '@num': '1',
    '@isCorrect': '0',
    '@numQuestion': '73',
    '@idQuestion': '22224',
    '@numTheme': '2',
    '#text': 'если правильный ответ 1,2 и 3'},
   {'@id': '99678',
    '@num': '2',
    '@isCorrect': '0',
    '@numQuestion': '73',
    '@idQuestion': '22224',
    '@numTheme': '2',
    '#text': 'если правильный ответ 1 и 3'},
   {'@id': '99679',
    '@num': '3',
    '@isCorrect': '0',
    '@numQuestion': '73',
    '@idQuestion': '22224',
    '@numTheme': '2',
    '#text': 'если правильный ответ 2 и 4'},
   {'@id': '99680',
    '@num': '4',
    '@isCorrect': '0',
    '@numQuestion': '73',
    '@idQuestion': '22224',
    '@numTheme': '2',
    '#tex

#### Building Answers DF

In [174]:
qq = []
for q in tqdm(dd['geetest']['test']['questions']['question']):
    aa = dict(text = q['text'],)
    for a in q['answers']['answer']:
        # print(a)
        # print (a['@num'], a['#text'], a['@isCorrect'])
        aa[a['@num']] = a['#text']
    correct_idx = [a['@num'] for a in q['answers']['answer'] if a['@isCorrect']=='1']
    try: 
        correct_idx = int(correct_idx[0])
    except IndexError:
        correct_idx = 0
        
    aa['correct'] = correct_idx

    qq.append(aa)
    # print(correct_idx)

  0%|          | 0/770 [00:00<?, ?it/s]

In [187]:
dfa0 = pd.DataFrame(qq)
print(dfa0.shape)
dfa0['lines'] = dfa0.text.apply(lambda x: len(x.split('\n')))
dfa0.head(3)

(770, 8)


,text,1,2,3,4,5,correct,6,lines
0,Везикулярное дыхание выслушивается на,вдохе,выдохе,вдохе и первой трети выдоха,вдохе и первых двух третях выдоха,протяжении всего вдоха и всего выдоха,3,NaN,1
1,При эмфиземе легких дыхание,везикулярное,везикулярное ослабленное,везикулярное усиленное,бронхиальное,саккадированное,2,NaN,1
2,Крепитация характерна для,эмфиземы легких,абсцесса легких,крупозной пневмонии,бронхиальной астмы,застоя крови в малом круге кровообращения,3,NaN,1


In [188]:
dfa0.correct.astype(str).value_counts()

1    204
5    194
2    140
3    122
4    107
0      3
Name: correct, dtype: int64

In [189]:
dfa0.lines.value_counts()

5    392
1    355
6     16
4      6
3      1
Name: lines, dtype: int64

In [134]:
# multiline question
dfa0.loc[730]

text       Для нормализации состава кишечной флоры при ди...
1                             если правильны ответы 1, 2 и 3
2                                если правильны ответы 1 и 3
3                                если правильны ответы 2 и 4
4                                    если правильный ответ 4
5                          если правильны ответы 1, 2, 3 и 4
correct                                                    5
6                                                        NaN
Name: 730, dtype: object

In [238]:
# cleanup and re-formatting
dfa = dfa0[dfa0.lines==1] \
    .drop(columns = ['6', 'lines'])\
    .rename(columns={'text': 'question'}) \
    .sample(frac=1,random_state=42,ignore_index=True) \
    .reset_index(drop=True)  # keep only 1-line questions, drop option 6(unused)
# dfa['q'] = dfa.question.str.replace('"', '').str.replace('ё', 'е')
dfa.shape

(355, 7)

In [192]:
# SAMPLE OF QUESTIONS
for i in range (0, len(dfa), 10):
    print (i, dfa.iloc[i]['question'])

0 Приналичие патологического зубца Q во II, III и aVF отведениях очаговые изменения локализуются
10 Основной симптом хронического бронхита, протекающего с преимущественным поражением слизистой оболочки мелких бронхов -это
20 При спонтанной стенокардии наиболее информативным диагностическим методом является
30 У больных артериальной гипертонией и сердечной недостаточностью для снижения АД целесообразно использовать
40 Несоответствие между потребностью миокарда в кислороде и его доставкой по коронарному руслу является причиной ишемии миокарда при
50 Общая длина толстой кишки
60 По данным эхокардиографии при митральном стенозе дилатируются следующие камеры сердца
70 Больная 33 лет обратилась в поликлинику с жалобами на боли за грудиной, чувство распирания в этой области.  Накануне вечером ела рыбу, при этом поперхнулась.  К врачу не обращалась.  Утром появились жалобы на затруднённое глотание, температура тела повысилась до 37,3 градусов С, появилось вздутие в области шеи и нижней половин

## PROMPT testing

In [6]:
row = df.loc[1]
template = """You need to find the best answer to the following question:
{}".
choose correct answer only for the four items below:
1. {}
2. {}
3. {}
4. {}
The number of the correct answer is:"""

rqst = template.format(*row[['question', '1', '2', '3', '4']])
print(rqst)

You need to find the best answer to the following question:
"Лёгочное сердце" может возникнуть при".
choose correct answer only for the four items below:
1. ишемической болезни сердца
2. гипертонической болезни
3. хронической обструктивной болезни лёгких
4. гипертиреозе
The number of the correct answer is:


In [7]:
row = df.loc[1]
template1 = """You are a medical doctor and need to answer which one of the four statements is correct:
1. {0} {1}".
2. {0} {2}".
3. {0} {3}".
4. {0} {4}".
The number of the correct statements is:"""

rqst1 = template1.format(*row[['question', '1', '2', '3', '4']])
print(rqst1)

You are a medical doctor and need to answer which one of the four statements is correct:
1. "Лёгочное сердце" может возникнуть при ишемической болезни сердца".
2. "Лёгочное сердце" может возникнуть при гипертонической болезни".
3. "Лёгочное сердце" может возникнуть при хронической обструктивной болезни лёгких".
4. "Лёгочное сердце" может возникнуть при гипертиреозе".
The number of the correct statements is:


In [114]:
row = df.loc[1]
template2 = """You need to pass a medical doctor exam and absolutely need to select correct answer from groups of the four statements, which has highest probability of being correct:
Statements:
1. В амбулаторных условиях день овуляции диагностируется с помощью измерения базальной температуры".
2. В амбулаторных условиях день овуляции диагностируется с помощью кольпоскопии".
3. В амбулаторных условиях день овуляции диагностируется с помощью гистероскопии".
4. В амбулаторных условиях день овуляции диагностируется с помощью кольпоцитологии".
Answer: 1
Statements:
1. {0} {1}".
2. {0} {2}".
3. {0} {3}".
4. {0} {4}".
Answer:"""

rqst1 = template2.format(*row[['question', '1', '2', '3', '4']])
print(rqst1)

You need to pass a medical doctor exam and absolutely need to select correct answer from groups of the four statements, which has highest probability of being correct:
Statements:
1. В амбулаторных условиях день овуляции диагностируется с помощью измерения базальной температуры".
2. В амбулаторных условиях день овуляции диагностируется с помощью кольпоскопии".
3. В амбулаторных условиях день овуляции диагностируется с помощью гистероскопии".
4. В амбулаторных условиях день овуляции диагностируется с помощью кольпоцитологии".
Answer: 1
Statements:
1. "Лёгочное сердце" может возникнуть при ишемической болезни сердца".
2. "Лёгочное сердце" может возникнуть при гипертонической болезни".
3. "Лёгочное сердце" может возникнуть при хронической обструктивной болезни лёгких".
4. "Лёгочное сердце" может возникнуть при гипертиреозе".
Answer:


In [132]:
row = df.loc[1]
template3 = """You are a medical doctor and absolutely need to answer which one of the four statements has highest probability of being correct:
1. {0} {1}".
2. {0} {2}".
3. {0} {3}".
4. {0} {4}".
The number of the most likely correct statement is:"""

rqst1 = template1.format(*row[['question', '1', '2', '3', '4']])
print(rqst1)

You are a medical doctor and need to answer which one of the four statements is correct:
1. "Лёгочное сердце" может возникнуть при ишемической болезни сердца".
2. "Лёгочное сердце" может возникнуть при гипертонической болезни".
3. "Лёгочное сердце" может возникнуть при хронической обструктивной болезни лёгких".
4. "Лёгочное сердце" может возникнуть при гипертиреозе".
The number of the correct statements is:


### single question tester

In [183]:
oa = OpenAI(engine="text-davinci-003")

In [184]:
q1 = """You need to pass a medical doctor exam and absolutely need to select correct answer from groups of the four statements, which has highest probability of being correct:
Statements:
1. В амбулаторных условиях день овуляции диагностируется с помощью измерения базальной температуры".
2. В амбулаторных условиях день овуляции диагностируется с помощью кольпоскопии".
3. В амбулаторных условиях день овуляции диагностируется с помощью гистероскопии".
4. В амбулаторных условиях день овуляции диагностируется с помощью кольпоцитологии".
Answer: 1
Statements:
1. Амфорическое дыхание наблюдается при абсцессе лёгкого".
2. Амфорическое дыхание наблюдается при бронхиальной астме".
3. Амфорическое дыхание наблюдается при бронхите".
4. Амфорическое дыхание наблюдается при очаговой пневмонии".
Answer:"""

In [185]:
ans = oa(q1)
ans

' 4'

### Prompt 3a

In [149]:
row = dfa1.iloc[1]
template3a = """You are a medical doctor and absolutely need to answer which one of the four statements has highest probability of being correct:
1. {0} {1}".
2. {0} {2}".
3. {0} {3}".
4. {0} {4}".
5. {0} {5}".
The number of the most likely correct statement is:"""

rqst1 = template3a.format(*row[['text', '1', '2', '3', '4', '5', '']])
print(rqst1)

You are a medical doctor and absolutely need to answer which one of the four statements has highest probability of being correct:
1. При эмфиземе легких дыхание везикулярное".
2. При эмфиземе легких дыхание везикулярное ослабленное".
3. При эмфиземе легких дыхание везикулярное усиленное".
4. При эмфиземе легких дыхание бронхиальное".
5. При эмфиземе легких дыхание саккадированное".
The number of the most likely correct statement is:


In [153]:
%%time 
log_rmt_2 =[]
for i in trange(20):
# for i in tqdm(range (df.shape[0])):
    row = dfa1.iloc[i]
    rqst = template3a.format(*row[['text', '1', '2', '3', '4', '5']])
    ans = oa(rqst)
    log_rmt_2 =[].append((i, row['text'], rqst, ans))
    print(i, row['correct'], ans, sep='\t')

  0%|          | 0/20 [00:00<?, ?it/s]

0 3  5. Везикулярное дыхание выслушивается на протяжении всего в
1 2  2. При эмфиземе легких дыхание везикулярное ослабленное".
2 3  2. Крепитация характерна для абсцесса легких".
3 4  3. Для крупозной пневмонии в разгар болезни характерен перк
4 4  5. Бронхиальное дыхание выслушивается на вдохе и первых дву
5 4  4. Амфорическое дыхание наблюдается при абсцессе легкого
6 2  4. Ослабление голосового дрожания характерно для очаговой пн
7 2  5. Усиление голосового дрожания типично для бронхиальной астм
8 4  3. Пульс  tardus, parvus характерен для недостаточности аортального кл
9 5  5. Наличие у больного положительного венного пульса характерно
10 2  4. Наиболее вероятная причина значительного расширения абс
11 4  5. Значительное расширение сердца влево и вниз наблюдается
12 2  3. Кровохарканье чаще всего наблюдается при недостаточнос
13 1  5. Систолическое дрожание над верхушкой сердца характерно д
14 5  3. Усиление систолического шума в области нижней трети груд
15 3  3. Односторонние бо

In [225]:
p3a = pd.DataFrame(log_rmt_2, columns = 'n a a0 q'.split(' '))
p3a.head()

,n,a,a0,q
0,0,3,5,Везикулярное дыхание выслушивается на протяжен...
1,1,2,2,При эмфиземе легких дыхание везикулярное ослаб...
2,2,3,2,"Крепитация характерна для абсцесса легких""."
3,3,4,3,Для крупозной пневмонии в разгар болезни харак...
4,4,4,5,Бронхиальное дыхание выслушивается на вдохе и ...


In [222]:
(p3a.a.astype(int) == p3a.a0.astype(int)).mean()

0.25

In [157]:
i = 2
row = dfa1.iloc[i]
rqst = template3a.format(*row[['text', '1', '2', '3', '4', '5']])
print(rqst)

You are a medical doctor and absolutely need to answer which one of the four statements has highest probability of being correct:
1. Крепитация характерна для эмфиземы легких".
2. Крепитация характерна для абсцесса легких".
3. Крепитация характерна для крупозной пневмонии".
4. Крепитация характерна для бронхиальной астмы".
5. Крепитация характерна для застоя крови в малом круге кровообращения".
The number of the most likely correct statement is:


### p3b (p3a with translation)

In [158]:
%%time 
log_rmt_2 =[]
for i in trange(20):
    row = dfa1.iloc[i]
    rqst = template3a.format(*row[['text', '1', '2', '3', '4', '5']])
    print(rqst)
    print('###')

  0%|          | 0/20 [00:00<?, ?it/s]

You are a medical doctor and absolutely need to answer which one of the four statements has highest probability of being correct:
1. Везикулярное дыхание выслушивается на вдохе".
2. Везикулярное дыхание выслушивается на выдохе".
3. Везикулярное дыхание выслушивается на вдохе и первой трети выдоха".
4. Везикулярное дыхание выслушивается на вдохе и первых двух третях выдоха".
5. Везикулярное дыхание выслушивается на протяжении всего вдоха и всего выдоха".
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has highest probability of being correct:
1. При эмфиземе легких дыхание везикулярное".
2. При эмфиземе легких дыхание везикулярное ослабленное".
3. При эмфиземе легких дыхание везикулярное усиленное".
4. При эмфиземе легких дыхание бронхиальное".
5. При эмфиземе легких дыхание саккадированное".
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need 

In [164]:
tr = """You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Vesicular breathing is heard on inspiration.
2. Vesicular breathing is heard on expiration.
3. Vesicular breathing is heard during inspiration and the first third of expiration.
4. Vesicular breathing is heard during inhalation and the first two-thirds of exhalation.
5. Vesicular breathing is heard throughout the entire inhalation and the entire exhalation.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. With emphysema, breathing is vesicular.
2. With emphysema, vesicular breathing is weakened.
3. With emphysema, vesicular breathing is increased.
4. With emphysema, bronchial breathing."
5. With emphysema, breathing is saccadic."
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Crepitus is characteristic of emphysema."
2. Crepitus is characteristic of a lung abscess."
3. Crepitus is characteristic of croupous pneumonia."
4. Crepitus is characteristic of bronchial asthma.
5. Crepitus is characteristic of stagnation of blood in the pulmonary circulation.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. For croupous pneumonia at the height of the disease, a dull tympanitis is characteristic on percussion.
2. For croupous pneumonia at the height of the disease, a boxed percussion sound is characteristic.
3. For croupous pneumonia at the height of the disease, a tympanic percussion sound is characteristic.
4. For croupous pneumonia at the height of the disease, a dull percussion sound is characteristic.
5. For croupous pneumonia at the height of the disease, a metallic percussion sound is characteristic.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Bronchial breathing is heard on inspiration.
2. Bronchial breathing is heard on exhalation.
3. Bronchial breathing is heard during inhalation and one third of exhalation.
4. Bronchial breathing is heard throughout the entire inhalation and the entire exhalation.
5. Bronchial breathing is heard during inhalation and the first two-thirds of exhalation.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Amphoric breathing is observed in focal pneumonia.
2. Amphoric breathing is observed in bronchitis.
3. Amphoric breathing is observed in bronchial asthma.
4. Amphoric breathing is observed with a lung abscess.
5. Amphoric breathing is observed in emphysema."
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Weakening of voice trembling is characteristic of bronchiectasis."
2. Weakening of voice trembling is characteristic of exudative pleurisy."
3. The weakening of voice trembling is characteristic of a lung abscess in the cavity stage.
4. Weakening of voice trembling is characteristic of focal pneumonia."
5. Weakening of voice trembling is characteristic of croupous pneumonia.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Increased vocal trembling is typical of hydrothorax."
2. Increased voice trembling is typical of a lung abscess in the cavity stage.
3. Increased voice trembling is typical of closed pneumothorax."
4. Increased voice trembling is typical of emphysema."
5. Increased voice trembling is typical for bronchial asthma.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Pulse tardus, parvus is characteristic of tricuspid valve insufficiency.
2. Pulse tardus, parvus is characteristic of mitral valve insufficiency.
3. Pulse tardus, parvus is characteristic of aortic valve insufficiency.
4. Pulse tardus, parvus is characteristic of stenosis of the aortic mouth.
5. Pulse tardus, parvus is characteristic of mitral stenosis.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. The presence of a positive venous pulse in a patient is characteristic of mitral valve insufficiency.
2. The presence of a positive venous pulse in a patient is characteristic of mitral stenosis.
3. The presence of a positive venous pulse in a patient is characteristic of aortic valve insufficiency.
4. The presence of a positive venous pulse in a patient is characteristic of stenosis of the aortic orifice.
5. The presence of a positive venous pulse in a patient is characteristic of tricuspid valve insufficiency.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. The most probable cause of a significant expansion of the absolute dullness of the heart is dilatation of the right atrium.
2. The most probable cause of a significant increase in the absolute dullness of the heart is dilatation of the right ventricle.
3. The most probable cause of a significant expansion of the absolute dullness of the heart is right ventricular hypertrophy.
4. The most probable cause of a significant increase in the absolute dullness of the heart is dilatation of the left ventricle.
5. The most probable cause of a significant expansion of the absolute dullness of the heart is left ventricular hypertrophy.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. A significant expansion of the heart to the left and down is observed with dilatation of the right atrium.
2. A significant expansion of the heart to the left and down is observed with dilatation of the right ventricle.
3. A significant expansion of the heart to the left and down is observed with dilatation of the left atrium.
4. A significant expansion of the heart to the left and down is observed with dilatation of the left ventricle.
5. A significant expansion of the heart to the left and down is observed with dilatation of the left atrium and right ventricle.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Hemoptysis is most often observed with mitral valve insufficiency.
2. Hemoptysis is most often observed with mitral stenosis.
3. Hemoptysis is most often observed with aortic valve insufficiency.
4. Hemoptysis is most often observed with stenosis of the aortic orifice."
5. Hemoptysis is most often observed with tricuspid valve insufficiency.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Systolic trembling over the apex of the heart is characteristic of mitral valve insufficiency.
2. Systolic trembling over the apex of the heart is characteristic of aortic valve insufficiency.
3. Systolic trembling over the apex of the heart is characteristic of mitral stenosis."
4. Systolic trembling over the apex of the heart is characteristic of aortic stenosis.
5. Systolic trembling over the apex of the heart is characteristic of tricuspid valve insufficiency.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Increased systolic murmur in the lower third of the sternum at the end of forced inspiration is typical of mitral valve insufficiency."
2. Increased systolic murmur in the lower third of the sternum at the end of forced inspiration is typical of mitral stenosis.
3. Increased systolic murmur in the lower third of the sternum at the end of forced inspiration is typical of aortic valve insufficiency.
4. Increased systolic murmur in the region of the lower third of the sternum at the end of forced inspiration is typical of aortic stenosis.
5. Increased systolic murmur in the lower third of the sternum at the end of forced inspiration is typical of tricuspid valve insufficiency.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Unilateral pain in the lumbar region is characteristic of acute glomerulonephritis.
2. Unilateral pain in the lumbar region is characteristic of acute cystitis.
3. Unilateral pain in the lumbar region is characteristic of acute pyelonephritis.
4. Unilateral pain in the lumbar region is characteristic of amyloidosis of the kidneys.
5. Unilateral pain in the lumbar region is characteristic of chronic glomerulonephritis.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Typical in ascites is a pronounced asymmetry of the abdomen. "
2. Typical in ascites venous pulse is characteristic of aortic valve insufficiency.
4. The presence of a positive venous pulse in a patient is characteristic of stenosis of the aortic orifice.
5. The presence of a positive venous pulse in a patient is characteristic of tricuspid valve insufficiency.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Albuminuria and hypoproteinemia combined with cylindruria and edema are characteristic of acute glomerulonephritis.
2. Albuminuria and hypoproteinemia combined with cylindruria and edema are characteristic of pyelonephritis."
3. Albuminuria and hypoproteinemia combined with cylindruria and edema are characteristic of nephrolithiasis.
4. Albuminuria and hypoproteinemia combined with cylindruria and edema are characteristic of cystitis.
5. Albuminuria and hypoproteinemia combined with cylindruria and edema are characteristic of nephroptosis.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. An indication for esophagoscopy is not a hiatal hernia."
2. Esophagitis is not an indication for esophagoscopy.
3. The indication for esophagoscopy is not cancer of the esophagus."
4. An indication for esophagoscopy is not a chest injury.
5. An indication for esophagoscopy is not varicose vei of the esophagus.
The number of the most likely correct statement is:
###
You are a medical doctor and absolutely need to answer which one of the four statements has the highest probability of being correct:
1. Of these diseases, a relative contraindication to esophagoscopy is varicose veins of the esophagus.
2. Of these diseases, epilepsy is a relative contraindication to esophagoscopy.
3. Of these diseases, a relative contraindication to esophagoscopy is an acute inflammatory disease of the tonsils, pharynx, larynx, bronchi.
4. Of these diseases, esophagitis is a relative contraindication to esophagoscopy.
5. Of these diseases, a relative contraindication to esophagoscopy is esophageal cancer.
The number of the most likely correct statement is:"""

In [223]:
qq_translated = (tr.split("###"))

In [168]:
log_rmt_3b = []
for i, rqst in enumerate(qq_translated):
    row = dfa1.iloc[i]
    ans = oa(rqst)
    log_rmt_3b =[].append((i, row['text'], rqst, ans))
    print(i, row['correct'], ans, sep = '\t')

0 3 3. Vesicular breathing is heard during inspiration and the first third of expiration.
1 2 2. With emphysema, vesicular breathing is weakened.
2 3 2. Crepitus is characteristic of a lung abscess.
3 4 3. For croupous pneumonia at the height of the disease, a tympanic percussion sound is characteristic.
4 4 5. Bronchial breathing is heard during inhalation and the first two-thirds of exhalation.
5 4 4. Amphoric breathing is observed with a lung abscess.
6 2 5. Weakening of voice trembling is characteristic of croupous pneumonia.
7 2 3. Increased voice trembling is typical of closed pneumothorax.
8 4 3. Pulse tardus, parvus is characteristic of aortic valve insufficiency.
9 5 3. The presence of a positive venous pulse in a patient is characteristic of aortic valve insufficiency.
10 2 5. The most probable cause of a significant expansion of the absolute dullness of the heart is left ventricular hypertrophy.
11 4 5. A significant expansion of the heart to the left and down is observed wi

In [226]:
zz = """0 3 3. Vesicular breathing is heard during inspiration and the first third of expiration.
1 2 2. With emphysema, vesicular breathing is weakened.
2 3 2. Crepitus is characteristic of a lung abscess.
3 4 3. For croupous pneumonia at the height of the disease, a tympanic percussion sound is characteristic.
4 4 5. Bronchial breathing is heard during inhalation and the first two-thirds of exhalation.
5 4 4. Amphoric breathing is observed with a lung abscess.
6 2 5. Weakening of voice trembling is characteristic of croupous pneumonia.
7 2 3. Increased voice trembling is typical of closed pneumothorax.
8 4 3. Pulse tardus, parvus is characteristic of aortic valve insufficiency.
9 5 3. The presence of a positive venous pulse in a patient is characteristic of aortic valve insufficiency.
10 2 5. The most probable cause of a significant expansion of the absolute dullness of the heart is left ventricular hypertrophy.
11 4 5. A significant expansion of the heart to the left and down is observed with dilatation of the left atrium and right ventricle.
12 2 3. Hemoptysis is most often observed with aortic valve insufficiency.
13 1 3. Systolic trembling over the apex of the heart is characteristic of mitral stenosis.
14 5 2. Increased systolic murmur in the lower third of the sternum at the end of forced inspiration is typical of mitral stenosis.
15 3 3. Unilateral pain in the lumbar region is characteristic of acute pyelonephritis.
16 3 4. The presence of a positive venous pulse in a patient is characteristic of stenosis of the aortic orifice.
17 1 1. Albuminuria and hypoproteinemia combined with cylindruria and edema are characteristic of acute glomerulonephritis.
18 4 3. The indication for esophagoscopy is not cancer of the esophagus.
19 3  5. Of these diseases, a relative contraindication to esophagoscopy is esophageal cancer."""

In [227]:
zz = [z.split('. ') for z in zz.split('\n')]

In [228]:
zz = [[*z[0].replace('  ',' ').split(' '), z[1]] for z in zz]

In [229]:
log_rmt_3b = zz

In [230]:
p3b = pd.DataFrame(log_rmt_3b, columns = 'n a a0 q'.split(' '))
p3b.head()

,n,a,a0,q
0,0,3,3,Vesicular breathing is heard during inspiratio...
1,1,2,2,"With emphysema, vesicular breathing is weakened."
2,2,3,2,Crepitus is characteristic of a lung abscess.
3,3,4,3,For croupous pneumonia at the height of the di...
4,4,4,5,Bronchial breathing is heard during inhalation...


In [235]:
(p3b.a.astype(int) == p3b.a0.astype(int)).mean()

0.25

In [236]:
(p3b.a.astype(int) == p3b.a0.astype(int))

0      True
1      True
2     False
3     False
4     False
5      True
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15     True
16    False
17     True
18    False
19    False
dtype: bool

In [237]:
(p3a.a.astype(int) == p3a.a0.astype(int))

0     False
1      True
2     False
3     False
4     False
5      True
6     False
7     False
8     False
9      True
10    False
11    False
12    False
13    False
14    False
15     True
16    False
17     True
18    False
19    False
dtype: bool

### With 1-shot

#### Prompt 3c

In [258]:
row = dfa.iloc[108]
template3a = """You are a medical doctor and absolutely need to answer which one of the five statements in each group is most correct:
Statements:
1. {0} {1}".
2. {0} {2}".
3. {0} {3}".
4. {0} {4}".
5. {0} {5}".
Answer number: {6}"""

rqst1 = template3a.format(*row[['question', '1', '2', '3', '4', '5', 'correct']])
print(rqst1)

You are a medical doctor and absolutely need to answer which one of the five statements in each group is most correct:
Statements:
1. У больного с грыжей пищеводного отверстия диафрагмы в последнее время изменилась клиническая картина заболевания: исчезла изжога и регургитация, появилась дисфагия при употреблении плотной пищи, пропал аппетит, больной стал терять в весе. Предполагаемый диагноз развитие рака пищевода".
2. У больного с грыжей пищеводного отверстия диафрагмы в последнее время изменилась клиническая картина заболевания: исчезла изжога и регургитация, появилась дисфагия при употреблении плотной пищи, пропал аппетит, больной стал терять в весе. Предполагаемый диагноз развитие рака желудка".
3. У больного с грыжей пищеводного отверстия диафрагмы в последнее время изменилась клиническая картина заболевания: исчезла изжога и регургитация, появилась дисфагия при употреблении плотной пищи, пропал аппетит, больной стал терять в весе. Предполагаемый диагноз ущемление грыжи".
4. У бо

In [ ]:
You are a medical doctor and absolutely need to answer which one of the five statements in each group is most correct:
Statements:
1. Характерным клиническим признаком хронического панкреатита является развитие сахарного диабета".
2. Характерным клиническим признаком хронического панкреатита является снижение функции внешней секреции (гипоферментемия)".
3. Характерным клиническим признаком хронического панкреатита является желтуха".
4. Характерным клиническим признаком хронического панкреатита является повышение активности аминотрансфераз".
5. Характерным клиническим признаком хронического панкреатита является гепатомегалия".
Answer number: 2

In [264]:
row = dfa.iloc[100]
template4 = """You are a medical doctor and absolutely need to answer which one of the five statements in each group is most correct:
Statements:
1. Сканирование печени наиболее результативно в диагностике очагового поражения печени".
2. Сканирование печени наиболее результативно в диагностике микронодулярного цирроза печени".
3. Сканирование печени наиболее результативно в диагностике гепатита".
4. Сканирование печени наиболее результативно в диагностике макронодулярного цирроза печени".
5. Сканирование печени наиболее результативно в диагностике застойного фиброза печени".
Answer number: 1
Statements:
1. Клетки, вырабатывающие гастрин, располагаются в кардиальном отделе желудка".
2. Клетки, вырабатывающие гастрин, располагаются в теле".
3. Клетки, вырабатывающие гастрин, располагаются в антральном отделе".
4. Клетки, вырабатывающие гастрин, располагаются в пилорическом отделе".
5. Клетки, вырабатывающие гастрин, располагаются в дне желудка".
Answer number: 3
Statements:
1. {0} {1}".
2. {0} {2}".
3. {0} {3}".
4. {0} {4}".
5. {0} {5}".
Answer number: """

rqst1 = template4.format(*row[['question', '1', '2', '3', '4', '5', 'correct']])
print(rqst1)

You are a medical doctor and absolutely need to answer which one of the five statements in each group is most correct:
Statements:
1. Сканирование печени наиболее результативно в диагностике очагового поражения печени".
2. Сканирование печени наиболее результативно в диагностике микронодулярного цирроза печени".
3. Сканирование печени наиболее результативно в диагностике гепатита".
4. Сканирование печени наиболее результативно в диагностике макронодулярного цирроза печени".
5. Сканирование печени наиболее результативно в диагностике застойного фиброза печени".
Answer number: 1
Statements:
1. Клетки, вырабатывающие гастрин, располагаются в кардиальном отделе желудка".
2. Клетки, вырабатывающие гастрин, располагаются в теле".
3. Клетки, вырабатывающие гастрин, располагаются в антральном отделе".
4. Клетки, вырабатывающие гастрин, располагаются в пилорическом отделе".
5. Клетки, вырабатывающие гастрин, располагаются в дне желудка".
Answer number: 3
Statements:
1. Характерным клиническим п

In [267]:
%%time 
# log_rmt_4 =[]
for i in trange(11,20):
# for i in tqdm(range (df.shape[0])):
    row = dfa.iloc[i]
    rqst = template4.format(*row[['question', '1', '2', '3', '4', '5']])
    ans = oa(rqst)
    log_rmt_4.append((i, row['question'], rqst, ans))
    print(i, row['correct'], ans, row['question'][:60], sep='\t')

  0%|          | 0/9 [00:00<?, ?it/s]

11	2	 1	Наиболее информативным для визуализации очага некроза в миок
12	4	 5	Диета у больных с обострением панкреатита в первые 2 дня
13	2	 5	При лечении желчнокаменной болезни при меняют
14	2	 2	При пролапсе митрального клапана аускультативная симптоматик
15	1	 2	Центральная вена печеночной дольки относится к
16	3	 3	При копрологическом исследовании при синдроме раздраженной т
17	1	 2	Для острого фибринозного перикардита не характерно
18	1	 2	Непрямая (неконъюгированная) гипербилирубинемия характерна д
19	1	 2	Наиболее редким осложнением застойного цирроза печени являет
CPU times: user 135 ms, sys: 0 ns, total: 135 ms
Wall time: 25 s


### Manual testing

In [270]:
row = dfa.iloc[0]
template3m = """You are a medical doctor and absolutely need to answer which one of the four statements has highest probability of being correct:
1. {0} {1}".
2. {0} {2}".
3. {0} {3}".
4. {0} {4}".
5. {0} {5}".
The number of the most likely correct statement is:

correct = {6}"""

rqst1 = template3m.format(*row[['question', '1', '2', '3', '4', '5', 'correct']])
print(rqst1)

You are a medical doctor and absolutely need to answer which one of the four statements has highest probability of being correct:
1. Приналичие патологического зубца Q во II, III и aVF отведениях очаговые изменения локализуются в задне-базальной области левого желудочка".
2. Приналичие патологического зубца Q во II, III и aVF отведениях очаговые изменения локализуются в верхне-боковой области левого желудочке".
3. Приналичие патологического зубца Q во II, III и aVF отведениях очаговые изменения локализуются в правом желудочке".
4. Приналичие патологического зубца Q во II, III и aVF отведениях очаговые изменения локализуются впередне-перегородочной области".
5. Приналичие патологического зубца Q во II, III и aVF отведениях очаговые изменения локализуются в области задней стенки".
The number of the most likely correct statement is:

correct = 5


## Load questions for benchmark

In [4]:
rumedbench_path = '../rumedbench'
pd.read_json(os.path.join(rumedbench_path, 'data/RuMedTest/private_test_v1.jsonl'), lines=True)

df = pd.read_json('rmtest_private_v1.jsonl', lines=True)
print(df.shape)
df.head(2)

(397, 6)


,idx,question,1,2,3,4
0,da4324a94e712843d32e5574474d2c15,"""Критическим"" считается уменьшение площади лев...",3 см2,2 см2,4 см2,1 см2 и менее
1,3b5567b9e918302ec2f79cad693d1676,"""Лёгочное сердце"" может возникнуть при",ишемической болезни сердца,гипертонической болезни,хронической обструктивной болезни лёгких,гипертиреозе


In [5]:
df.columns

Index(['idx', 'question', '1', '2', '3', '4'], dtype='object')

In [32]:
df.head()

,idx,question,1,2,3,4
0,da4324a94e712843d32e5574474d2c15,"""Критическим"" считается уменьшение площади лев...",3 см2,2 см2,4 см2,1 см2 и менее
1,3b5567b9e918302ec2f79cad693d1676,"""Лёгочное сердце"" может возникнуть при",ишемической болезни сердца,гипертонической болезни,хронической обструктивной болезни лёгких,гипертиреозе
2,2dc1358c1f3288d26443e253143d3d0c,Четвертым приёмом Леопольда-Левицкого определя...,позиция плода,высота стояния дна матки,отношение предлежащей части плода ко входу в м...,вид плода
3,07af0fd92b3fa342845dbed30985cdd0,Амфорическое дыхание наблюдается при,абсцессе лёгкого,бронхиальной астме,бронхите,очаговой пневмонии
4,7721678440a7020dd02b1b06527f6e79,Бактериоскопическое исследование мокроты для о...,пять раз,три раза,двукратно,однократно


## experiment runner

In [203]:
oa = OpenAI(engine="text-davinci-003")

In [ ]:
log_rmt_1 =[]

In [ ]:
%%time 
# for i in trange(15):
for i in tqdm(range (df.shape[0])):
    row = df.iloc[i]
    rqst = template3.format(*row[['question', '1', '2', '3', '4']])
    ans = oa(rqst)
    log.append((i, row['idx'], row['question'], rqst, ans))

In [187]:
[l[4] for l in log_rmt_1[:15]]

[' 4. "Критическим" считается уменьшение площади левого атриове',
 ' 3. "Лёгочное сердце" может возникнуть при хронической обс',
 ' 3. Четвертым приёмом Леопольда-Левицкого определяется отнош',
 ' 4. Амфорическое дыхание наблюдается при очаговой пневмони',
 ' 2. Бактериоскопическое исследование мокроты для определения ки',
 ' 1. Болезненность при пальпации в левом реберно-позвоночном',
 ' 2. Большой родничок закрывается в возрасте 9-18 месяцев".',
 ' 3. Бронхиальная обструкция выявляется с помощью Бронхоскоп',
 ' 2. Бронхиальное дыхание выслушивается на протяжении всего в',
 ' 1. В амбулаторных условиях день овуляции диагностируется с',
 ' 4. В диагностике детского церебрального паралича на 1 году жи',
 ' 2. В общей лечебной сети должны выполняться следующие м',
 ' 3. Ведущим клиническим признаком нефротического синдрома яв',
 ' 3. Витилиго обычно встречается у людей в любом периоде ж',
 ' 3. Возбудителем внебольничной пневмонии чаще всего являетс']

In [208]:
len(log_rmt)

397

In [188]:
#saving submit
with open("rmtest_gpt3_1002.jsonl", 'w') as f:
    for a in log:
        ans = a[4].split()[0].strip('.')
        line = f'{{"idx": "{a[1]}", "answer": "{ans}"}}'
        f.write(line + '\n')
        # print (line)

In [211]:
# Saving whole log
with open("logs/rmtest_gpt3_1002_log.json", "w") as f:
    json.dump(log_rmt, f)

In [410]:
log_rmt[0]

[0,
 'da4324a94e712843d32e5574474d2c15',
 '"Критическим" считается уменьшение площади левого атриовентрикулярного отверстия до',
 'You are a medical doctor and absolutely need to answer which one of the four statements has highest probability of being correct:\n1. "Критическим" считается уменьшение площади левого атриовентрикулярного отверстия до 3 см2".\n2. "Критическим" считается уменьшение площади левого атриовентрикулярного отверстия до 2 см2".\n3. "Критическим" считается уменьшение площади левого атриовентрикулярного отверстия до 4 см2".\n4. "Критическим" считается уменьшение площади левого атриовентрикулярного отверстия до 1 см2 и менее".\nThe number of the most likely correct statement is:',
 ' 4. "Критическим" считается уменьшение площади левого атриове']

In [412]:
[l[4] for l in log_rmt[40:55]]

[' 3. Крепитация характерна для абсцесса лёгких".',
 ' 4. Лейкоцитурия наиболее характерный симптом при гломеруло',
 ' 4. Местными признаками хронического тонзиллита является гип',
 ' 3. Методом лабораторного обследования, имеющим решающее з',
 ' 1. Молочные зубы у большинства детей появляются в возрасте:',
 ' 4. Морфологическим субстратом хронической болезни почек я',
 ' 3. На дому у пациента диагностирован экссудативный плеврит, ',
 ' 4. На ЭКГ интервалы между комплексами QRS соседних циклов от',
 ' 4. На ЭКГ продолжительность интервала PQ больше 0,20 с. Это ха',
 ' 1. Наиболее важным рентгенологическим признаком остеохондр',
 ' 1. Наиболее типичным клиническим симптомом крапивницы явл',
 ' 1. Наиболее характерным симптомом аллергического конъюнкт',
 ' 2. Наиболее часто встречающимся вариантом эктопической бе',
 ' 3. Наиболее частое осложнение беременности в первом триместре 1',
 ' 3. К фактору риска рака лёгкого относится курение".']

# RuMedDaNet

In [276]:
oa = OpenAI(engine="text-davinci-003")

In [243]:
rumedbench_path = '../rumedbench'
df_dev = pd.read_json(os.path.join(rumedbench_path, 'data/RuMedDaNet/dev_v1.jsonl'), lines=True)
print(df_dev.shape)
df_dev.head(2)

(256, 4)


,context,question,answer,pairID
0,"Апоплексия яичника — кровоизлияние в яичник, с...",Зачастую причиной апоплексии яичника становитс...,да,1b6019e574f3767872a97386743eacd3
1,Дерматиты — воспалительные реакции кожи в отве...,Заболевания из группы дерматитов связаны с вос...,нет,61895c2af233c6623e0500ee59ed1c1c


In [223]:
dfdev.loc[11].question

'Шоколад и кофе повышают тонус нижнего пищеводного сфинктера?'

## templates

In [258]:
row = df_dev.loc[6]
template_dn_1 = """You are a medical doctor and need to pass exam. The question is:
{}
You should answer only yes or no.
The answer is """

rqst = template_dn_1.format(*row[['question']])
print(rqst)

You are a medical doctor and need to pass exam. The question is:
Все стероидные гормоны синтезируются из холестерина?
You should answer only yes or no.
The answer is 


In [259]:
rqst = template_dn_1.format(*row[['question']])
ans = oa(rqst)
ans, row.answer

(' No.', 'да')

In [261]:
row = df_dev.loc[6]
template_dn_2 = """Imagine that you are a medical doctor and know everything about medicine and need to pass exam. The question is:
{}
You should answer only yes or no.
The answer is """

rqst = template_dn_2.format(*row[['question']])
print(rqst)

Imagine that you are a medical doctor and know everything about medicine and need to pass exam. The question is:
Все стероидные гормоны синтезируются из холестерина?
You should answer only yes or no.
The answer is 


In [262]:
rqst = template_dn_2.format(*row[['question']])
ans = oa(rqst)
ans, row.answer

(' No.', 'да')

#### t3

In [398]:
row = df_dev.loc[9]
template_dn_3 = """Imagine that you are a medical doctor and know everything about medicine and need to pass exam. 
The context is: {}
The question is: {}
You should answer only yes or no.
The answer is """

rqst = template_dn_3.format(*row[['context','question']])
print(rqst)

Imagine that you are a medical doctor and know everything about medicine and need to pass exam. 
The context is: Мочевой пузырь — у взрослого человека лежит в малом тазу позади лобкового симфиза. Объем 0,5 литров. Имеет дно, тело и верхушку. Нижний отдел, суживаясь, образует шейку пузыря, переходящую в мочеиспускательный канал. Между вершиной и дном лежит тело пузыря. Пустой мочевой пузырь покрыт брюшиной сверху, немного с боков и сзади. При его наполнении мочой он округляется, верхушка поднимается вверх, и брюшина покрывает его спереди, с боков и особенно заднюю поверхность. Дно пузыря у мужчин сзади и снизу прилежит к предстательной железе и семенным пузырькам, сзади — к ампуле прямой кишки. У женщин к влагалищу и матке.
The question is: Дно мочевого пузыря у женщин прилегает к предстательной железе?
You should answer only yes or no.
The answer is 


In [384]:
row = df_dev.loc[51]

rqst = template_dn_3.format(*row[['context','question']])
print(row['question'])
ans = oa(rqst)
ans, row.answer

Разноцветный лишай вызывает Candida albicans?


(' No.', 'нет')

In [385]:
%%time 
df_ = df_dev.copy()
# idx_range_ = trange(20)
idx_range_ = trange(len(df_))
template_ = template_dn_3
idx_ = 'pairID'

log_dn2 = []
log_ = log_dn2

for i in idx_range_:
    row = df_.iloc[i]
    rqst = template_.format(*row[['context','question']])
    ans = oa(rqst).strip()
    log_.append((i, row[idx_], row['question'], rqst, ans))
    # print(ans)

  0%|          | 0/256 [00:00<?, ?it/s]

CPU times: user 2.96 s, sys: 341 ms, total: 3.3 s
Wall time: 13min 15s


In [387]:
len(log_dn1), len(log_dn2)

(256, 256)

In [388]:
df_ = df_dev.copy()
df_['a1'] = [l[-1].split()[0].strip('.').lower().replace('yes', "да").replace('no', 'нет') for l in log_dn1]
assert df_.a1.nunique() == 2
df_['r1'] = df_.a1 == df.answer

In [390]:
df_['a2'] = [l[-1].split()[0].strip('.').lower().replace('yes', "да").replace('no', 'нет') for l in log_dn2]
assert df_.a2.nunique() == 2
df_['r2'] = df_.a2 == df.answer

In [391]:
df_.head(10)

,context,question,answer,pairID,a1,r1,a2,r2
0,"Апоплексия яичника — кровоизлияние в яичник, с...",Зачастую причиной апоплексии яичника становитс...,да,1b6019e574f3767872a97386743eacd3,да,True,да,True
1,Дерматиты — воспалительные реакции кожи в отве...,Заболевания из группы дерматитов связаны с вос...,нет,61895c2af233c6623e0500ee59ed1c1c,нет,True,нет,True
2,Цинк пиритион активированный обладает противов...,Pityrosporum ovale и Pityrosporum orbiculare у...,да,07696e23dbc26edd1361cf839c0965d4,да,True,да,True
3,L-треонин в присутствии пиридоксина (витамина ...,Для превращения L-треонина в глицин и ацетальд...,нет,46b80fa75fcfdec8e18c80818e7727a2,нет,True,нет,True
4,Синдром слабости синусового узла (СССУ) связан...,СССУ происходит вследствие ослабевания или пре...,да,9c94af23ae52b30418646c6f82681970,нет,False,да,True
5,"Эксгибиционизм — сексуальная девиация, при кот...",Эксгибиционизм это вариант нормы?,нет,481d0be5e2ce023b6b08edf90ad1bc59,нет,True,нет,True
6,Человек получает все свои стероидные гормоны и...,Все стероидные гормоны синтезируются из холест...,да,e56c247fd06d1de59cdc7d6eb9237d35,нет,False,нет,False
7,Всё пространство внутри кости заполнено костны...,Красный костный мозг в основном состоит из жир...,нет,f38b78ee0ffb8fd004abc062f5b5186b,нет,True,нет,True
8,"Сердце — полый мышечный орган, нагнетающий кро...",По внешнему виду сердце напоминает форму конуса?,да,7b1410347182b6d9a01fcae7504136f7,да,True,да,True
9,Мочевой пузырь — у взрослого человека лежит в ...,Дно мочевого пузыря у женщин прилегает к предс...,нет,ea56913cf815c54dff3b7b765c383360,да,False,нет,True


In [394]:
df_.r1.mean(), df_.r2.mean()

(0.671875, 0.8359375)

In [397]:
# ERRORS in answers with context
for idx, row in df_[~df_.r2].iterrows():
    print (idx)
    print(*row[['context','question','answer']], sep = '\n', end = '\n\n')

6
Человек получает все свои стероидные гормоны из холестерина. Два класса стероидных гормонов синтезируются в коре надпочечников: минералкортикоиды, которые регулируют всасывание неорганических ионов в почках, и глюкокортикоиды, которые помогают регулировать глюконеогенез и уменьшают воспалительную реакцию. Половые гормоны производятся в половых клетках мужчин и женщин и в плаценте. Среди них прогестерон, который регулирует женский репродуктивный цикл, андрогены (например, тестостерон) и эстрогены (эстрадиол), которые влияют на развитие вторичных половых признаков у мужчин и женщин соответственно. Стероидные гормоны оказывают эффект при очень низких концентрациях и поэтому синтезируются в относительно малых количествах. По сравнению с желчными солями для продукции стероидных гормонов потребляется относительно мало холестерина.
Все стероидные гормоны синтезируются из холестерина?
да

16
С возрастом компактное вещество утолщается, перекладины губчатого вещества становятся крупнее. Мозгов

In [341]:
(df_.answer == df_.a1).mean()

0.671875

In [342]:
df_.a1.value_counts()

да     134
нет    122
Name: a1, dtype: int64

In [413]:
for q in range (0, len(df_), 20):
    print (q, df_.iloc[q:q+20].r2.mean())

0 0.9
20 0.85
40 0.75
60 0.85
80 0.7
100 0.9
120 0.85
140 0.9
160 0.75
180 0.9
200 0.8
220 0.85
240 0.875


In [414]:
df_.iloc[40:60]

,context,question,answer,pairID,a1,r1,a2,r2
40,"Коклюш — острая инфекционная болезнь, относяща...",Болезнь передаётся воздушно-капельным путём?,да,917e79c795f732256dcdbf0562496a5d,да,True,да,True
41,"Кандидоз (кандидамикоз, молочница) — группа за...",Возбудителм группы заболеваний является бактер...,нет,ce5f4992b380f5d73b2f6dd44d82f9f9,да,False,нет,True
42,"Подобно окситоцину, карбетоцин избирательно св...",Окситоцин и карбетоцин имеют сходное действие ...,да,7c76ca98d5ae9f55cac0753f428d421d,нет,False,да,True
43,Эффективен у взрослых пациентов с гиперхолесте...,Классификация по Фредриксону характеризует ста...,нет,845036554853af98600f3784c1622bea,да,False,нет,True
44,"Возбудитель — актиномицеты, грамположительные ...",Возбудителем являются грамположительные аэробн...,да,9617d41bfcfedc5f02173a5c772cf8d0,да,True,нет,False
45,"Возбудитель — хламидия, которая имеет оболочку...",Инфекционный возбудитель этой болезни называет...,нет,7572687c0e297cd74a44052f349d7ac3,нет,True,нет,True
46,"Сибирская язва является острым, особо опасным ...",Можно ли разделывая свежее мясо коровы заразит...,да,83d29f7ec67c62752e9b6e675834727a,нет,False,нет,False
47,При характеристике базального ритма необходимо...,Изменения базального ритма характеризуется тол...,нет,ff9759657679e91c241fd9267ac5be6d,нет,True,нет,True
48,Признаки дефицита. Снижение или отсутствие вса...,Мегалобластная анемия возникает вследствие деф...,да,a411da0c6cc8b355e0405b33f712d439,да,True,да,True
49,Понижает моторику и тонус гладкой мускулатуры ...,Данный препарат понижает тонус скелетной муску...,нет,8e5f404d7f817858dc401b97449d8d66,нет,True,нет,True


In [377]:
len(log_dn1)

256

## private test

In [416]:
rumedbench_path = '../rumedbench'
df_priv = pd.read_json(os.path.join(rumedbench_path, 'data/RuMedDaNet/private_test_v1.jsonl'), lines=True)
print(df_priv.shape)
df_priv.head(2)

(512, 3)


,pairID,context,question
0,8c4f70416beeda1f12e00f5104d9d908,Природа полос поглощения в ик-области связана ...,Возможности ИК-спектроскопии позволяют анализи...
1,f96a9c5168641bf26d1e44de63010c3e,Глюкокортикоиды оказывают разностороннее влиян...,Глюкокортикоиды не встречаются в тканях печени?


In [425]:
%%time 
df_ = df_priv.copy()
# idx_range_ = trange(20)
idx_range_ = trange(len(df_))
template_ = template_dn_3
idx_ = 'pairID'

log_dn_priv_1 = []
log_ = log_dn_priv_1

for i in idx_range_:
    ans = None
    row = df_.iloc[i]
    rqst = template_.format(*row[['context','question']])
    while ans is None:
        try:
            ans = oa(rqst).strip()
        except(openai.error.RateLimitError,):
            print('_RLError_')
            time.sleep(1)
    log_.append((i, row[idx_], row['question'], rqst, ans))
    print(ans, row['question'][:120])

  0%|          | 0/512 [00:00<?, ?it/s]

Yes. Возможности ИК-спектроскопии позволяют анализировать вещества в кристаллическом состоянии?
No. Глюкокортикоиды не встречаются в тканях печени?
No. Проламины и глютелины широко распространенны в органах и тканях животных (сыворотка крови, молоко, яичный белок, мышцы)?
Yes. Поликлиники могут долечивать больных после выписки из стационара?
_RLError_
No. Отобранный при данной процедуре объём крови замещается 5%-ым раствором марганцовки?
_RLError_
No. Трупные яды образуются в результате присоединения карбоксильной группы к белковым аминокислотам?
No. Здесь описан процесс анаэробного дыхания?
_RLError_
No. Аскорбатоксидаза относится к ферментам содержащим молибден?
No. Центр голода находится в вентромедиальных ядрах подбугорья?
Yes. Возможна ли атрофия мочевого пузыря?
No. Только при необходимости длительной транспортировки более шести суток транспортные шины необходимо укреплять гипсом?
Yes. Биофлавоноиды подавляют выработку гиалуронидазы?
No. Медиастинит - это аутоиммунное заболевание

In [426]:
len(log_dn_priv_1)

512

In [462]:
log_dn_priv_1[0]

(0,
 '8c4f70416beeda1f12e00f5104d9d908',
 'Возможности ИК-спектроскопии позволяют анализировать вещества в кристаллическом состоянии?',
 'Imagine that you are a medical doctor and know everything about medicine and need to pass exam. \nThe context is: Природа полос поглощения в ик-области связана с колебательными переходами и изменением колебательных состояний ядер, входящих в молекулу поглощающего вещества. Поэтому поглощением в ИК-области обладают молекулы, дипольные моменты которых изменяются при возбуждении колебательных движений ядер. Область применения ИК-спектроскопии аналогична, но более широка, чем УФ-метода. ИК-спектр однозначно характеризует всю структуру молекулы, включая незначительные ее изменения. Важные преимущества данного метода — высокая специфичность, объективность полученных результатов, возможность анализа веществ в кристаллическом состоянии.\nThe question is: Возможности ИК-спектроскопии позволяют анализировать вещества в кристаллическом состоянии?\nYou should 

In [434]:
clean_dn = lambda x: x.split()[0].strip('.').lower().replace('yes', 'да').replace('no', 'нет')

In [463]:
print('Imagine that you are a medical doctor and know everything about medicine and need to pass exam. \nThe context is: Природа полос поглощения в ик-области связана с колебательными переходами и изменением колебательных состояний ядер, входящих в молекулу поглощающего вещества. Поэтому поглощением в ИК-области обладают молекулы, дипольные моменты которых изменяются при возбуждении колебательных движений ядер. Область применения ИК-спектроскопии аналогична, но более широка, чем УФ-метода. ИК-спектр однозначно характеризует всю структуру молекулы, включая незначительные ее изменения. Важные преимущества данного метода — высокая специфичность, объективность полученных результатов, возможность анализа веществ в кристаллическом состоянии.\nThe question is: Возможности ИК-спектроскопии позволяют анализировать вещества в кристаллическом состоянии?\nYou should answer only yes or no.\nThe answer is ')

Imagine that you are a medical doctor and know everything about medicine and need to pass exam. 
The context is: Природа полос поглощения в ик-области связана с колебательными переходами и изменением колебательных состояний ядер, входящих в молекулу поглощающего вещества. Поэтому поглощением в ИК-области обладают молекулы, дипольные моменты которых изменяются при возбуждении колебательных движений ядер. Область применения ИК-спектроскопии аналогична, но более широка, чем УФ-метода. ИК-спектр однозначно характеризует всю структуру молекулы, включая незначительные ее изменения. Важные преимущества данного метода — высокая специфичность, объективность полученных результатов, возможность анализа веществ в кристаллическом состоянии.
The question is: Возможности ИК-спектроскопии позволяют анализировать вещества в кристаллическом состоянии?
You should answer only yes or no.
The answer is 


In [438]:
#saving submit
with open("submits/rmdn_gpt3_1402.jsonl", 'w') as f:
    for a in log_dn_priv_1:
        ans = clean_dn(a[4])
        line = f'{{"pairID": "{a[1]}", "answer": "{ans}"}}'
        f.write(line + '\n')
        # print (line)

In [439]:
# Saving whole log
with open("logs/rmdanet_priv_gpt3_1402.json", "w") as f:
    json.dump(log_dn_priv_1, f)

# RuMedNLI

In [6]:
oa = OpenAI(engine="text-davinci-003")

In [7]:
rumedbench_path = '../rumedbench'
df_dev_nli = pd.read_json(os.path.join(rumedbench_path, 'data/RuMedNLI/dev_v1.jsonl'), lines=True)
print(df_dev_nli.shape)
df_dev_nli.head(2)

(1395, 4)


,ru_sentence1,ru_sentence2,gold_label,pairID
0,"В анамнезе нет тромбозов или ТГВ, никогда не б...",Пациент страдает стенокардией,entailment,1892c19a-66c7-11e7-be8f-f45c89b91419
1,"В анамнезе нет тромбозов или ТГВ, никогда не б...",Перенес несколько эпизодов ТЭЛА,contradiction,1892c578-66c7-11e7-97cf-f45c89b91419


In [8]:
df_dev_nli.gold_label.unique()

array(['entailment', 'contradiction', 'neutral'], dtype=object)

In [9]:
df_dev_nli.loc[11].ru_sentence1, df_dev_nli.loc[11].ru_sentence2

('Во время госпитализации у пациента постепенно усиливалась одышка, что потребовало выполнения процедуры неинвазивной вентиляции лёгких с положительным давлением, а затем маска без ребризера.',
 'У пациента отёк легких.')

## templates

#### t1

In [10]:
row = df_dev_nli.loc[44]
template_nli_1 = """You are a medical doctor and need to pass exam. You are given two statements:
The first statement is absolutely correct and should be the basis for your answer: "{}"
The second statement is "{}". 
You should answer if the second statement is 'entailment', 'contradiction', or 'neutral'.
The answer is """

rqst = template_nli_1.format(*row[['ru_sentence1', 'ru_sentence2']])
print(rqst)
print(row['gold_label'])

You are a medical doctor and need to pass exam. You are given two statements:
The first statement is absolutely correct and should be the basis for your answer: "Сахарный диабет II типа."
The second statement is "Инсулин не требуется.". 
You should answer if the second statement is 'entailment', 'contradiction', or 'neutral'.
The answer is 
neutral


In [12]:
row = df_dev_nli.loc[1]
rqst = template_nli_1.format(*row[['ru_sentence1', 'ru_sentence2']])
print(row)
ans = oa(rqst)
ans, row.gold_label

ru_sentence1    В анамнезе нет тромбозов или ТГВ, никогда не б...
ru_sentence2                      Перенес несколько эпизодов ТЭЛА
gold_label                                          contradiction
pairID                       1892c578-66c7-11e7-97cf-f45c89b91419
Name: 1, dtype: object


(" 'neutral'.", 'contradiction')

#### t2

In [13]:
row = df_dev_nli.loc[32]
template_nli_2 = """
Statement 1: "{}"
Statement 2: "{}"
Answer: "{}"
"""

rqst = template_nli_2.format(*row[['ru_sentence1', 'ru_sentence2', 'gold_label']])
print(rqst)
print(row['gold_label'])


Statement 1: "КТ головы показала небольшое правое височное внутрипаренхиматозное кровоизлияние 2х2 см, на повторной КТ головы осталось без изменений."
Statement 2: "у пациента было гипертоническое кровотечение"
Answer: "neutral"

neutral


In [11]:
row = df_dev_nli.loc[44]
template_nli_2 = """You are a medical doctor and need to pass exam. You are given two statements and need to answer how the second statement relates to the first statement. Possible answers are 'entailment', 'contradiction', or 'neutral'
Statement 1: "В анамнезе нет тромбозов или ТГВ, никогда не было болей в груди до случаев недельной давности."
Statement 2: "Пациент страдает стенокардией"
Answer: "entailment"
Statement 1: "В течение последней недели стал более сонливым и трудно возбудимым."
Statement 2: "В последнюю неделю он был менее внимателен"
Answer: "entailment"
Statement 1: "КТ головы показала небольшое правое височное внутрипаренхиматозное кровоизлияние 2х2 см, на повторной КТ головы осталось без изменений."
Statement 2: "у пациента было гипертоническое кровотечение"
Answer: "neutral"
Statement 1: "Рентгенограмма чистая, не похоже наличие инфекции."
Statement 2: "Рентген грудной клетки выявил инфильтраты"
Answer: "contradiction"
Statement 1: "КТ головы показала небольшое правое височное внутрипаренхиматозное кровоизлияние 2х2 см, на повторной КТ головы осталось без изменений."
Statement 2: "у пациента была нормальная КТ головы"
Answer: "contradiction"
Statement 1: "{}"
Statement 2: "{}"
Answer:"""

rqst = template_nli_2.format(*row[['ru_sentence1', 'ru_sentence2']])
print(rqst)

You are a medical doctor and need to pass exam. You are given two statements and need to answer how the second statement relates to the first statement. Possible answers are 'entailment', 'contradiction', or 'neutral'
Statement 1: "В анамнезе нет тромбозов или ТГВ, никогда не было болей в груди до случаев недельной давности."
Statement 2: "Пациент страдает стенокардией"
Answer: "entailment"
Statement 1: "В течение последней недели стал более сонливым и трудно возбудимым."
Statement 2: "В последнюю неделю он был менее внимателен"
Answer: "entailment"
Statement 1: "КТ головы показала небольшое правое височное внутрипаренхиматозное кровоизлияние 2х2 см, на повторной КТ головы осталось без изменений."
Statement 2: "у пациента было гипертоническое кровотечение"
Answer: "neutral"
Statement 1: "Рентгенограмма чистая, не похоже наличие инфекции."
Statement 2: "Рентген грудной клетки выявил инфильтраты"
Answer: "contradiction"
Statement 1: "КТ головы показала небольшое правое височное внутрипар

In [349]:
row = df_dev_nli.loc[111]
rqst = template_nli_2.format(*row[['ru_sentence1', 'ru_sentence2']])
print(row)
ans = oa(rqst)
ans, row.gold_label

ru_sentence1    В общей сложности получила 4 л физраствора в р...
ru_sentence2                             Пациент был реанимирован
gold_label                                             entailment
pairID                       1893d24a-66c7-11e7-9a61-f45c89b91419
Name: 111, dtype: object


(' "entailment"', 'entailment')

In [299]:
row = df_dev_nli.loc[1]
rqst = template_nli_1.format(*row[['ru_sentence1', 'ru_sentence2']])
print(row)
ans = oa(rqst)
ans, row.gold_label

ru_sentence1    В анамнезе нет тромбозов или ТГВ, никогда не б...
ru_sentence2                      Перенес несколько эпизодов ТЭЛА
gold_label                                          contradiction
pairID                       1892c578-66c7-11e7-97cf-f45c89b91419
Name: 1, dtype: object


(" 'neutral'.", 'contradiction')

### test run NLI

In [371]:
%%time 
df_ = df_dev_nli
idx_range_ = trange(40, 60)
# idx_range_ = trange(len(df_))
template_ = template_nli_2
idx_ = 'pairID'

# log_nli_1 = []
# log_ = log_nli_1

for i in idx_range_:
    row = df_.iloc[i]
    rqst = template_.format(*row[['ru_sentence1', 'ru_sentence2']])
    ans = oa(rqst).strip()
    log_.append((i, row[idx_], rqst, ans))
    print(i, ans, row.gold_label)

  0%|          | 0/20 [00:00<?, ?it/s]

40 "contradiction" contradiction
41 "neutral" neutral
42 "entailment" entailment
43 "contradiction" contradiction
44 "entailment" neutral
45 "entailment" entailment
46 "contradiction" contradiction
47 "neutral" neutral
48 "entailment" entailment
49 "contradiction" contradiction
50 "entailment" neutral
51 "neutral" entailment
52 "neutral" contradiction
53 "neutral" neutral
54 "entailment" entailment
55 "contradiction" contradiction
56 "neutral" neutral
57 "entailment" entailment
58 "neutral" contradiction
59 "neutral" neutral
CPU times: user 355 ms, sys: 35.7 ms, total: 390 ms
Wall time: 50.3 s


In [355]:
len(log_)

20

In [372]:
a24 = {l[0]:l[-1].split()[0].strip('""').lower() for l in log_}

In [368]:
d4 = df_dev_nli.iloc[40:60].copy()
d4['a1'] = a14
d4['r1'] = d4.a1 == d4.gold_label
assert d4.a1.nunique() == 3
# = [l[-1].split()[0].strip('.').lower().replace('yes', "да").replace('no', 'нет') for l in log_]
# d4['a1'] = a1 = df_dev

In [373]:
d4['a2'] = a24
d4['r2'] = d4.a2 == d4.gold_label
assert d4.a2.nunique() == 3
# = [l[-1].split()[0].strip('.').lower().replace('yes', "да").replace('no', 'нет') for l in log_]
# d4['a1'] = a1 = df_dev

In [440]:
d4.sample()

,ru_sentence1,ru_sentence2,gold_label,pairID,a1,r1,a2,r2
44,Сахарный диабет II типа.,Инсулин не требуется.,neutral,18933a8a-66c7-11e7-a132-f45c89b91419,entailment,False,entailment,False


### long dev run NLI

In [12]:
%%time 
df_ = df_dev_nli
# idx_range_ = trange(40, 60)
idx_range_ = trange(len(dev_nli))
template_ = template_nli_1
idx_ = 'pairID'

# log_nli_1 = []
# log_ = log_nli_1

for i in idx_range_:
    row = df_.iloc[i]
    rqst = template_.format(*row[['ru_sentence1', 'ru_sentence2']])
    ans = oa(rqst).strip()
    log_.append((i, row[idx_], rqst, ans))
    print(i, ans, row.gold_label)

NameError: name 'dev_nli' is not defined

In [286]:
(df.answer == df.a1).mean()

0.671875

In [287]:
df.a1.value_counts()

да     134
нет    122
Name: a1, dtype: int64

In [305]:
for q in range (0, len(df_), 20):
    print (q, df.iloc[q:q+20].r1.mean())

0 0.8
20 0.65
40 0.45
60 0.8
80 0.65
100 0.8
120 0.55
140 0.7
160 0.75
180 0.5
200 0.65
220 0.7
240 0.75


In [32]:
%%time 
df_ = df_dev_nli
idx_range_ = trange(40, 60)
# idx_range_ = trange(len(dev_nli))
template_ = template_nli_1
idx_ = 'pairID'

log_nli_1 = []
log_ = log_nli_1

for i in idx_range_:
    row = df_.iloc[i]
    rqst = template_.format(*row[['ru_sentence1', 'ru_sentence2']])
    ans = oa(rqst).strip()
    log_.append((i, row[idx_], rqst, ans))
    print(i, ans, row.gold_label)

  0%|          | 0/20 [00:00<?, ?it/s]

40 'contradiction'. contradiction
41 'neutral'. neutral
42 'entailment'. entailment
43 'contradiction'. contradiction
44 'contradiction'. neutral
45 'neutral'. entailment
46 'neutral'. The second statement does not necessarily contradict or entail the first statement. It is possible that the patient had episodes of hyperglycemia that required large doses of insulin, but their blood sugar levels were still within normal range. contradiction
47 'neutral'. The second statement does not contradict the first, but it does not necessarily follow from it either. neutral
48 'contradiction'. entailment
49 'neutral'. contradiction
50 'neutral'. neutral
51 'neutral'. entailment
52 'neutral'. contradiction
53 'neutral'. The second statement does not provide any information that would either confirm or contradict the first statement. neutral
54 'neutral'. entailment
55 'contradiction'. contradiction
56 'neutral'. neutral
57 'neutral'. entailment
58 'neutral'. contradiction
59 'neutral'. The second s

In [18]:
len(log_nli_2)

20

In [27]:
(df_.loc[40:59, 'gold_label'] == [l[-1].strip('"') for l in log_nli_2]).mean()

0.75

In [45]:
import re
def clean_nli(x):
    x = x.split()[0]
    x = re.sub(r'[^a-z]', '', x)
    return x

In [47]:
(df_.loc[40:59, 'gold_label'] == [clean_nli(l[-1]) for l in log_nli_1]).mean()

0.5

In [30]:
row

ru_sentence1    На [**2810-12-14**] также был удален желудочко...
ru_sentence2      Отсутствие в анамнезе неврологических симптомов
gold_label                                          contradiction
pairID                       18935380-66c7-11e7-9dbb-f45c89b91419
Name: 52, dtype: object

In [29]:
row = df_dev_nli.loc[52]
template_nli_1 = """You are a medical doctor and need to pass exam. You are given two statements:
The first statement is absolutely correct and should be the basis for your answer: "{}"
The second statement is "{}". 
You should answer if the second statement is 'entailment', 'contradiction', or 'neutral'.
The answer is """

rqst = template_nli_1.format(*row[['ru_sentence1', 'ru_sentence2']])
print(rqst)
print(row['gold_label'])

You are a medical doctor and need to pass exam. You are given two statements:
The first statement is absolutely correct and should be the basis for your answer: "На [**2810-12-14**] также был удален желудочковый дренаж."
The second statement is "Отсутствие в анамнезе неврологических симптомов". 
You should answer if the second statement is 'entailment', 'contradiction', or 'neutral'.
The answer is 
contradiction


### NLI private_test_v1

In [71]:
oa = OpenAI(engine="text-davinci-003")

In [13]:
rumedbench_path = '../rumedbench'
df_priv_nli = pd.read_json(os.path.join(rumedbench_path, 'data/RuMedNLI/private_test_v1.jsonl'), lines=True)
print(df_priv_nli.shape)
df_priv_nli.head(2)

(1536, 3)


,pairID,ru_sentence1,ru_sentence2
0,001703e1a833361eeaf94d40fd470e82,Боли в области поясничного отдела позвоночника...,"Жалуется на сухой кашель, насморк, боли в горле."
1,004a0e3e3fd89e410e503dbfd368a39b,цитология [**2134-1-12**] - норма; маммография...,проведено ультразвуковое исследование органов ...


In [20]:
len(log_nli_priv_0)

1273

In [21]:
# Saving whole log
with open("logs/rmnli_priv_gpt3_0-521_784_plus.json", "w") as f:
    json.dump(log_nli_priv_0, f)

In [19]:
%%time 
df_ = df_priv_nli
# idx_range_ = trange(40, 60)
# idx_range_ = trange(df_.shape[0])
idx_range_ = trange(521)
template_ = template_nli_2
idx_ = 'pairID'

# log_nli_priv_0 = []
log_ = log_nli_priv_0

for i in idx_range_:
    row = df_.iloc[i]
    rqst = template_.format(*row[['ru_sentence1', 'ru_sentence2']])
    ans = oa(rqst).strip()
    log_.append((i, row[idx_], rqst, ans))
    print(i, ans)

  0%|          | 0/521 [00:00<?, ?it/s]

0 "neutral"
1 "neutral"
2 "contradiction"
3 "neutral"
4 "entailment"
5 "entailment"
6 "entailment"
7 "neutral"
8 "entailment"
9 "entailment"
10 "neutral"
11 "neutral"
12 "neutral"
13 "contradiction"
14 "contradiction"
15 "neutral"
16 "neutral"
17 "neutral"
18 "entailment"
19 "neutral"
20 "contradiction"
21 "neutral"
22 "neutral"
23 "contradiction"
24 "entailment"
25 "entailment"
26 "neutral"
27 "entailment"
28 "entailment"
29 "entailment"
30 "entailment"
31 "neutral"
32 "neutral"
33 "neutral"
34 "neutral"
35 "neutral"
36 "neutral"
37 "entailment"
38 "contradiction"
39 "neutral"
40 "contradiction"
41 "entailment"
42 "neutral"
43 "neutral"
44 "entailment"
45 "contradiction"
46 "contradiction"
47 "entailment"
48 "entailment"
49 "contradiction"
50 "neutral"
51 "neutral"
52 "neutral"
53 "neutral"
54 "entailment"
55 "entailment"
56 "neutral"
57 "neutral"
58 "entailment"
59 "neutral"
60 "entailment"
61 "entailment"
62 "entailment"
63 "entailment"
64 "entailment"
65 "neutral"
66 "contradiction

In [26]:
dn=pd.DataFrame(log_nli_priv_0)

In [41]:
dn.shape

(1560, 4)

In [52]:
dn.columns = "id pairID q gold_label".split()

# dn = dn.drop_duplicates(subset=['pairID',])
dn.gold_label = dn.gold_label.str.strip('"')
dn.shape

(1536, 4)

In [55]:
dn.gold_label.value_counts()

entailment       646
neutral          635
contradiction    255
Name: gold_label, dtype: int64

In [57]:
dn = df_.merge(dn[['pairID', 'gold_label']], on='pairID')
dn.shape

(1536, 4)

In [58]:
dn.sample()

,pairID,ru_sentence1,ru_sentence2,gold_label
851,852b030c4dd5d8a7b4e21dc06f4882ed,касательно этиологии нового эпизода фибрилляци...,ранее была госпитализация по поводу БСК,neutral


In [60]:
#saving submit
with open("submits/rmnli_gpt3_1502.jsonl", 'w') as f:
    for i, row in dn.iterrows():
        line = f'{{"pairID": "{row["pairID"]}", "gold_label": "{row["gold_label"]}"}}'
        f.write(line + '\n')
        # print (line)

In [61]:
dn.to_pickle('logs/rmnli_priv_gpt3_1502.pd.pickle')